In [45]:
!pip install python-dotenv
!pip install openai

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [46]:
import os
import csv
import base64
from dotenv import load_dotenv
from openai import OpenAI
import time
import random 

In [ ]:
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
def read_image_urls(csv_path):
    """Read image URLs into a dict: {image_number: image_url}"""
    urls = {}
    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            if len(row) >= 2:
                img_num = row[0].strip()
                img_url = row[1].strip()
                urls[img_num] = img_url
    return urls


In [49]:
def read_questions(csv_path):
    """Read questions into a dict: {image_number: question_text}"""
    questions = {}
    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) >= 2:
                num, question = row[0].strip(), row[1].strip()
                questions[num] = question
    return questions

In [50]:
def read_options(csv_path):
    """Read options into a dict: {question_number: [option1, option2, ...]}"""
    options = {}
    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) >= 2:
                num, option = row[0].strip(), row[1].strip()
                options.setdefault(num, []).append(option)
    return options

In [ ]:
def ask_gpt4o_about_image_url(image_url, question, options):
    """Ask GPT-4o-mini about an image via URL, structured for multimodal input."""

    question_text = (
        f"Question: {question}\n"
        f"Options: {options if options else 'No options provided'}"
    )

    messages = [
        {
            "role": "system",
            "content": (
                "Answer as a Latvian traffic rules expert. "
                "Analyze the image and return only the correct answer to the question. "
                "Answer should fully match one of the provided options. "
                "Base your answer solely on the image content. "
                "If options exist, choose one. "
            )
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": question_text},

                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_url,
                        "detail": "high"
                    },
                },
            ],
        }
    ]

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            max_tokens=500
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
def main():
    images_csv = "image_URLs.csv"
    questions_csv = "questions.csv"
    options_csv = "options.csv"

    results_all = []

    image_urls = read_image_urls(images_csv)
    questions = read_questions(questions_csv)
    options = read_options(options_csv)
    
    all_image_nums = sorted(list(set(image_urls.keys()) & set(questions.keys())))
    
    random.shuffle(all_image_nums)

    print(f"\n==============================")
    print(f"Starting sequential processing of {len(all_image_nums)} images.")
    print(f"==============================\n")
    count = 1
    for img_num in all_image_nums:
        image_url = image_urls[img_num]
        question = questions[img_num]
        answer_options = options.get(img_num, [])

        print(f"Processing image {img_num}... ({count}/{len(all_image_nums)})")
        answer = ask_gpt4o_about_image_url(image_url, question, answer_options)
        
        count += 1

        results_all.append({
            "image_number": img_num,
            "question": question,
            "options": answer_options,
            "model_answer": answer
        })

        print(f"Image {img_num}: {answer}\n")
        time.sleep(3)

    with open("results_OpenAI_image_URL.csv", "w", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(
            f,
            fieldnames=["image_number", "question", "options", "model_answer"]
        )
        writer.writeheader()
        for result in results_all:
            img_num = result["image_number"]
            writer.writerow({
                "image_number": img_num,
                "question": questions.get(img_num, ""),
                "options": ";".join(options.get(img_num, [])),
                "model_answer": result["model_answer"]
            })

    print("Saved all results to: results_OpenAI_image_URL.csv")

In [53]:
if __name__ == "__main__":
    main()


Starting sequential processing of 100 images.

Processing image 10... (1/100)
Image 10: Only straight and to the left.

Processing image 64... (2/100)
Image 64: I will cross first.

Processing image 63... (3/100)
Image 63: That a solid line is ahead.

Processing image 14... (4/100)
Image 14: You will give way to the pedestrian.

Processing image 31... (5/100)
Image 31: You will give way to both drivers.

Processing image 21... (6/100)
Image 21: Only the white car.

Processing image 80... (7/100)
Image 80: It is forbidden.

Processing image 86... (8/100)
Image 86: 'Red vehicle.'

Processing image 66... (9/100)
Image 66: I will give way to both vehicles.

Processing image 51... (10/100)
Image 51: I will give way to the tram and the automobile.

Processing image 50... (11/100)
Image 50: I will give way to the tram.

Processing image 18... (12/100)
Image 18: The driver of the yellow car.

Processing image 72... (13/100)
Image 72: 'Immediately stop. '

Processing image 23... (14/100)
Image